# How does climate change feel around the globe?
# The final project from Spiced Academy
# Notebook for data cleaning

Check analysis.ipynb for data analysis.

Check vizzes.ipynb for generating visualizations.

## Contents

[The questions](#questions)

[Importing libraries and packages](#import)
    
[Cleaning OECD data: Population exposure to extreme temperatures](#clean_oecd_1)

[Cleaning OECD data: Historical populations of countries](#clean_oecd_2)

[Cleaning UN data: GDP per capita](#clean_un)

[Cleaning NASA data: temperature anomaly](#clean_nasa)

## The questions <a id='questions'></a>

In this project, I attempt to answer these pressing questions related to periods of heat which are ever more frequent in virtually every part of the wolrd:
1. What percentage of people has direct experience with extreme heat?
2. Does this number change over time?
3. How much is it related to the global temperature anomaly?
4. Is there a clear link between wealth and heat exposure of populations?

## Importing libraries and packages <a id='import'></a>

In [40]:
import pandas as pd   # df workflow
import pycountry   # to get a dict between country names and alpha3 codes

## Cleaning OECD data: Population exposure to extreme temperatures <a id='clean_oecd_1'></a>

We load the data about population exposures and perform cleaning for further analysis and integration with other datasets.

Years included: 1979 - 2021.

We will be interested mainly in four measures, describing the population exposure, although there is more measures in the dataset.

- 'HD_TN_POP_IND' gives the exposures to hot summer days (over 35 °C) AND tropical nights (over 20 °C).

- 'HD_POP_IND' is the exposure to hot summer days.

- 'TN_POP_IND' is the exposure to tropical nights.

- 'ID_POP_IND' is the exposure to icing days (doesn't exceed 0 °C).

In [41]:
df_exp = pd.read_csv('../data/oecd_population_exposure_to_extreme_temp.csv')

Basic exploration. The table contains a lot of unnecessary information which I will drop or filter out.

In [42]:
df_exp.head()

In [43]:
df_exp.info()

In [44]:
df_exp['REF_AREA'].unique()

In [45]:
df_exp['MEASURE'].unique()

Choose only the relevant columns.

Reference area for individual countries (contains also provinces and greater regions of the world), measure (later we choose population exposure), duration (how many weeks per year did the phenomenon last), time period (from which year is the data), observed value.

In [46]:
df_exp = df_exp[['REF_AREA','MEASURE','DURATION','TIME_PERIOD','OBS_VALUE']]

Rename the columns to standard format, drop possible empty spaces.

In [47]:
df_exp.columns = df_exp.columns.str.lower().str.strip()

When reference area contains provinces, the abbreviation of a respective country ends with numerical characters. We are not interested in provinces and they occupy big part of the data frame, drop these lines.

In [48]:
df_exp = df_exp.loc[~(df_exp['ref_area'].str.endswith(('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')))]
df_exp.reset_index(drop=True, inplace=True)

Some tropical countries reported zero population exposures. This data is most likely flawed and we drop these countries.

In [49]:
df_exp = df_exp[~df_exp['ref_area'].isin(['BDI', 'RWA', 'GNQ', 'PRI', 'BHS'])]
df_exp.reset_index(drop=True, inplace=True)

Now let us create a column with country names based on the alpha3 abbreviations.

First, make a dictionary to translate countries alpha3 abbreviations.

In [50]:
country_alpha3 = {}
for country in pycountry.countries:
    country_alpha3[country.alpha_3] = country.name

Apply to create a new column with country names.

In [51]:
df_exp['country'] = df_exp['ref_area'].map(country_alpha3)   # new column with country names
df_exp = df_exp[['ref_area', 'country', 'measure', 'duration', 'time_period', 'obs_value']]   # reorder columns

Save the data frame into a file.

In [52]:
df_exp.to_csv('../exported_dfs/exposures_all_durations.csv', index=False)

The data frame before, with separate durations, is useful for example for investigating trends in greater detail.

**For some purposes, we will be using the sum over all durations. Let us make such a grouped data frame.**

This can be done only for measure expressing percentages of population, so we will filter tha data frame for those first.

In [53]:
df_exp_summed = df_exp[df_exp['measure'].isin(['HD_POP_IND', 'TN_POP_IND', 'HD_TN_POP_IND', 'ID_POP_IND'])]

Now it is reasonable to rename the column 'obs_value' to 'exposure'.

In [54]:
df_exp_summed.rename(columns={'obs_value':'exposure'}, inplace=True)

We can group and sum up exposures over all durations.

In [55]:
df_exp_summed = df_exp_summed.groupby(['ref_area', 'country', 'measure', 'time_period'], as_index=False)['exposure'].sum()

The result sometimes slightly exceeds 100 % (up to 104.5 %). In the case, round the summed exposure to 100 %.

In [56]:
df_exp_summed.loc[df_exp_summed['exposure']>100, 'exposure'] = 100

Save the data frame into a file.

In [57]:
df_exp_summed.to_csv('../exported_dfs/exposures_summed.csv', index=False)

## Cleaning OECD data: Historical populations of countries <a id='clean_oecd_2'></a>

We load the data about population of countries in the previous years and perform cleaning for further analysis and integration with other datasets.

Years included: 1950 - 2021.

In [58]:
df_pop = pd.read_csv('../data/oecd_historical_population_data.csv')

Basic exploration. Also here, only relevant columns will be filtered.

In [59]:
df_pop.head()

In [60]:
df_pop.info()

Choose only the relevant columns.

Reference area (country), measure (population), sex and age (to filter fot total values later), time period (year of the observation), observed value.

In [61]:
df_pop = df_pop.drop(['DATAFLOW', 'TIME_HORIZ', 'OBS_STATUS', 'UNIT_MULT', 'DECIMALS'], axis='columns')

Rename the columns to standard format, drop possible empty spaces.

In [62]:
df_pop.columns = df_pop.columns.str.lower().str.strip()
df_pop.rename(columns={'obs_value':'population'}, inplace=True)   # more informative name

Keep only rows for total population across sexes and ages and drop the unnecessary columns after.

In [63]:
df_pop = df_pop[(df_pop['measure']=='POP') & (df_pop['unit_measure']=='PS') & (df_pop['sex']=='_T') &\
                (df_pop['age']=='_T')]
df_pop.reset_index(drop=True, inplace=True)
df_pop = df_pop.drop(['measure', 'unit_measure', 'sex', 'age'], axis='columns')

Save the data frame into a file.

In [64]:
df_pop.to_csv('../exported_dfs/populations_clean.csv', index=False)

## Cleaning UN data: GDP per capita <a id='clean_un'></a>

We load the data about GDP per capita of countries in the previous years and perform cleaning for further analysis and integration with other datasets.

Years included: 2017 - 2021.


In [65]:
df_gdp = pd.read_csv('../data/un_gdp_per_capita_current_prices.csv')

Basic exploration. The 'Item' column is not relevant.

In [66]:
df_gdp.head()

In [67]:
df_gdp.info()

Choose only the relevant columns and rename.

In [68]:
df_gdp = df_gdp.drop('Item', axis='columns')
df_gdp.columns = ['country', 'time_period', 'gdp']

Change 'gdp' column type to float.

In [69]:
df_gdp['gdp'] = df_gdp['gdp'].astype('float')

We are using alpha3 abbreviations of countries to merge different data frames but these are not present in the UN data.

Create an inversed dictionary (see cleaning OECD data) to translate countries in the UN dataset to alpha3 abbreviations.

In [70]:
country_alpha3_inversed = {}
for country in pycountry.countries:
    country_alpha3_inversed[country.name] = country.alpha_3

Some country names in the UN data frame are different than in the pycountries library, we need to rename them.

In [71]:
old_keys = ['China', 'United Kingdom', 'Turkey', "Korea, Democratic People's Republic of", "Korea, Republic of",\
           'Venezuela, Bolivarian Republic of', 'Bolivia, Plurinational State of',\
            'Congo, The Democratic Republic of the', 'Tanzania, United Republic of', 'Moldova, Republic of',\
            'North Macedonia']
new_keys = ['China (mainland)', 'United Kingdom of Great Britain and Northern Ireland', 'Türkiye',\
           "Democratic People's Republic of Korea", "Republic of Korea", 'Venezuela (Bolivarian Republic of)',\
           'Bolivia (Plurinational State of)', 'Democratic Republic of the Congo',\
            'United Republic of Tanzania: Mainland', 'Republic of Moldova', 'Republic of North Macedonia']

for key in range(len(new_keys)):
    old_key = old_keys[key]
    new_key = new_keys[key]
    country_alpha3_inversed[new_key] = country_alpha3_inversed.pop(old_key)

Create the column with alpha3 abbreviations.


In [72]:
df_gdp['ref_area'] = df_gdp['country'].map(country_alpha3_inversed)
df_gdp = df_gdp[['ref_area', 'country', 'time_period', 'gdp']]   # redorder columns

Save the data frame into a file.

In [73]:
df_gdp.to_csv('../exported_dfs/gdp_clean.csv', index=False)

## Cleaning NASA data: temperature anomaly <a id='clean_nasa'></a>

We load the data about temperature anomaly and perform cleaning for further analysis and integration with other datasets.

Years included: 1880 - 2023.

Monthly temperature anomaly, compared with the mean temperature from 1951-1980.


In [74]:
df_temp_anomaly = pd.read_csv('../data/nasa_global_mean_temperature_anomaly.csv', skiprows=1)

In [75]:
df_temp_anomaly.head()

In [76]:
df_temp_anomaly.info()

In [77]:
df_temp_anomaly.describe()

Choose only relevant columns (year and monthly anomalies) and rename.


In [78]:
df_temp_anomaly = df_temp_anomaly.iloc[:,0:13]
df_temp_anomaly.columns = df_temp_anomaly.columns.str.lower().str.strip()

Choose only the relevant years (rows).

In [79]:
df_temp_anomaly = df_temp_anomaly[df_temp_anomaly['year'].between(1979,2021)]
df_temp_anomaly = df_temp_anomaly.reset_index(drop=True)

Change data type to floats to perform calculations.

In [80]:
df_temp_anomaly[df_temp_anomaly.columns[1:13]] = df_temp_anomaly[df_temp_anomaly.columns[1:13]].astype(float)

Add a column with average anomalies for every year and keep only these two columns.


In [81]:
df_temp_anomaly['avg_anomaly'] = df_temp_anomaly.iloc[:, 1:].mean(axis=1)
df_temp_anomaly = df_temp_anomaly[['year', 'avg_anomaly']]

Save the data frame into a file.

In [82]:
df_temp_anomaly.to_csv('../exported_dfs/temp_anomaly_clean.csv', index=False)